In [ ]:
import torch 
import torchvision 
import torchvision.transforms as transforms
from PIL import Image, ImageFilter
import cv2
import numpy as np

In [ ]:
tensor_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=tensor_transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tensor_transform)
train_data, test_data

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()
net.load_state_dict(torch.load('Baseline.pth'))
net.train()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
#Gaussian blur

def gaussianblur(images):
    blurred_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)
        
        blurred_image = pil_image.filter(ImageFilter.GaussianBlur(radius=0.1))
        
        transform2 = transforms.ToTensor()
        tensor_blurred_image = transform2(blurred_image)
        blurred_images_batch.append(tensor_blurred_image)
    blurred_images_batch = torch.stack(blurred_images_batch)
    return blurred_images_batch

In [ ]:
#Median filtering

def medianfilter(images):
    median_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)
        
        median_image = pil_image.filter(ImageFilter.MedianFilter(size = 1))
        
        transform2 = transforms.ToTensor()
        tensor_median_image = transform2(median_image)
        median_images_batch.append(tensor_median_image)
    median_images_batch = torch.stack(median_images_batch)
    return median_images_batch

In [ ]:
#Non Local Means Denoising

def nlmdenoising(images):
    nlm_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)

        h=10
        np_image = np.array(pil_image)
        denoised_image = cv2.fastNlMeansDenoisingColored(np_image, None, h , h , templateWindowSize = 7, searchWindowSize = 21)
        nlm_image = Image.fromarray(denoised_image)
        
        transform2 = transforms.ToTensor()
        tensor_nlm_image = transform2(nlm_image)
        nlm_images_batch.append(tensor_nlm_image)
    nlm_images_batch = torch.stack(nlm_images_batch)
    return nlm_images_batch

In [ ]:
#FGSM

epsilon = 0.15
net.eval()

def FGSM_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def denorm(batch, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    if isinstance(mean, list):
        mean = torch.tensor(mean)
    if isinstance(std, list):
        std = torch.tensor(std)
    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        images, labels = data
        images.requires_grad = True
        outputs = net(images)
        loss = criterion(outputs, labels)
        net.zero_grad()
        loss.backward()
        data_grad = images.grad.data
        data_denorm = denorm(images)
        perturbed_images = FGSM_attack(images, epsilon, data_grad)
        perturbed_images_normalized = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(perturbed_images)
        net.train()
        optimizer.zero_grad()
        outputs = net(perturbed_images_normalized)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    scheduler.step()
print('Finished Training')

In [ ]:
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        images, labels = data
        optimizer.zero_grad()
        
        preprocessed_images = 
        outputs = net(preprocessed_images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    scheduler.step()
print('Finished Training')

In [ ]:
PATH = './fgsm_trained.pth'
torch.save(net.state_dict(), PATH)